### This script will combine coverage metrics across multiple sequencing experiments (RNA-seq, Ribo-seq, WES, WGS, etc) together with Ribo-seq translation quantification across multiple replicates if available. 

In [2]:

# Author: Tamara Ouspenskaia
# Date: 01/14/2020
# Objective: Combine sequencing read coverage and translation quantification for all variants



In [5]:
import pandas as pd
import numpy as np
from functools import reduce


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [7]:
# snvs.analysis is an output file from variant_pipeline.sh

annot = pd.read_csv('../snvs.analysis', header=0, sep='\t')

In [8]:
annot.columns.values

array(['ORF_ID', 'variant_ORF_id', 'trans_variant_id', 'wt', 'mut',
       'variant_qc', 'ORF_type', 'prot_mut'], dtype=object)

In [9]:
annot_keep = annot[['ORF_ID','variant_ORF_id','variant_qc', 'ORF_type','prot_mut']]

In [13]:
# Incorporate translation quantification if you want to consider translation levels in variant priotarization 

TPM1 = '/ahg/regevdata/projects/Ribo-seq/MHC-I/tpm/vartpm/Mel11_1.tpm'
TPM3 = '/ahg/regevdata/projects/Ribo-seq/MHC-I/tpm/vartpm/Mel11_3.tpm'
TPM4 = '/ahg/regevdata/projects/Ribo-seq/MHC-I/tpm/vartpm/Mel11_4.tpm'

SAMPLES = {
    'Mel11_1' : TPM1,
    'Mel11_3' : TPM3,
    'Mel11_4' : TPM4
}

In [14]:
def getTPM(path, annot, name):
    TPM = pd.read_csv(
        path,
        sep='\t',
        header=0,
        usecols=['ORF_ID','TPM'])
    names = TPM.columns.values
    newNames = ['ORF_ID', name + '_' + names[1]]
    TPM.columns = newNames
    return(TPM)

In [15]:
def main():
    annot = annot_keep
    TPM = [annot]
    for sample, path in SAMPLES.items():
        TPM.append(
            getTPM(path, annot, sample))
    df_final = reduce(lambda left,right: pd.merge(left,right,on='ORF_ID'), TPM)
    return(df_final)

In [16]:
out= main()

In [17]:
out.head()

,ORF_ID,variant_ORF_id,variant_qc,ORF_type,prot_mut,Mel11_1_TPM,Mel11_3_TPM,Mel11_4_TPM
0,TNFRSF18|ENST00000328596.10_2_1:1138973-114195...,TNFRSF18|ENST00000328596.10_2_1:1138973-114195...,PASS,canonical,E64K,0.0,0.000000,0.0
1,TNFRSF18|ENST00000379268.6_2_1:1139226-1141951...,TNFRSF18|ENST00000379268.6_2_1:1139226-1141951...,PASS,canonical,E64K,0.0,0.046895,0.0
2,TNFRSF18|ENST00000486728.5_1_1:1139226-1140990...,TNFRSF18|ENST00000486728.5_1_1:1139226-1140990...,PASS,canonical_extended,E41K,0.0,0.051843,0.0
3,TNFRSF18|ENST00000379265.5_1_1:1139226-1141951...,TNFRSF18|ENST00000379265.5_1_1:1139226-1141951...,PASS,canonical,E64K,0.0,0.048298,0.0
4,C1QTNF12|ENST00000330388.2_1_1:1178525-1179833...,C1QTNF12|ENST00000330388.2_1_1:1178525-1179833...,PASS,within,A62T,0.0,0.000000,0.0


In [19]:
def mut_loc(var_orf_id):
    everything = var_orf_id.split('|')
    var_loc = '_'.join(everything[-4:])
    return var_loc

In [20]:
out['mut_loc'] = out['variant_ORF_id'].apply(mut_loc)

In [21]:
out.to_csv('../snvs.brief.analysis', header=True, index=None, sep='\t')

In [42]:
#ANNOTATION is the snvs.brief.analysis file generated above. 
#snvs.read_counts.txt is output from snv_coverage.sh


OUT = '../snv_coverage_across_samples.txt'
ANNOTATION = '../snvs.brief.analysis'
MEL11_1 = '../snv_coverage/Mel11_1/snvs.read_counts.txt'
MEL11_3 = '../snv_coverage/Mel11_3/snvs.read_counts.txt'
MEL11_4 = '../snv_coverage/Mel11_4/snvs.read_counts.txt'
NorWES = '../snv_coverage/Nor_origWESbam/snvs.read_counts.txt'
TumWES = '../snv_coverage/Tum_origWESbam/snvs.read_counts.txt'
TumRNA = '../snv_coverage/Tum_origRNAbam/snvs.read_counts.txt'
Mel11_PBMCs_WGS = '../snv_coverage/Mel11_PBMCs_WGS/snvs.read_counts.txt'
cellline_WGS = '../snv_coverage/Mel11_tumor_WGS/snvs.read_counts.txt'

SAMPLES = {
    'mel11.1': MEL11_1,
    'mel11.3': MEL11_3,
    'mel11.4': MEL11_4,
    '13240-011.Nor.WES': NorWES,
    '13240-011.Tum.WES': TumWES,
    '13240-011.Tum.RNA': TumRNA,
    'Mel11_PBMCs_WGS' : Mel11_PBMCs_WGS,
    'cellline_WGS' : cellline_WGS
}


In [43]:
def importAnnot(path):
    annot = pd.read_csv(
        path,
        sep='\t',
        header=0,
        index_col=False)
    return(annot)

In [44]:
def getCoverage(path, annot, name):
    coverage = pd.read_csv(
        path,
        sep='\t',
        header=0,
        usecols=['variant_ORF_id','wt_cov','mut_cov'])
    names = coverage.columns.values
    newNames = [names[0], name + '_' + names[1], name + '_' + names[2]]
    coverage.columns = newNames
    return(coverage)

In [45]:
def main():
    annot = importAnnot(ANNOTATION)
    coverage = [annot]
    for sample, path in SAMPLES.items():
        coverage.append(
            getCoverage(path, annot, sample))
    df_final = reduce(lambda left,right: pd.merge(left,right,on='variant_ORF_id'), coverage)
    df_final.to_csv(
            OUT,
            sep='\t',
            header=True,
            index=False)
    return(df_final)

In [46]:
out= main()

In [47]:
out.head(10)

,ORF_ID,variant_ORF_id,variant_qc,ORF_type,prot_mut,Mel11_1_TPM,Mel11_3_TPM,Mel11_4_TPM,mut_loc,mel11.1_wt_cov,mel11.1_mut_cov,mel11.3_wt_cov,mel11.3_mut_cov,mel11.4_wt_cov,mel11.4_mut_cov,13240-011.Nor.WES_wt_cov,13240-011.Nor.WES_mut_cov,13240-011.Tum.WES_wt_cov,13240-011.Tum.WES_mut_cov,13240-011.Tum.RNA_wt_cov,13240-011.Tum.RNA_mut_cov,Mel11_PBMCs_WGS_wt_cov,Mel11_PBMCs_WGS_mut_cov,cellline_WGS_wt_cov,cellline_WGS_mut_cov
0,TNFRSF18|ENST00000328596.10_2_1:1138973-114195...,TNFRSF18|ENST00000328596.10_2_1:1138973-114195...,PASS,canonical,E64K,0.000000,0.000000,0.000000,1_1140870_C_T,0,0,0,0,0,0,108,0,46,35,7,0,35,0,1,50
1,TNFRSF18|ENST00000379268.6_2_1:1139226-1141951...,TNFRSF18|ENST00000379268.6_2_1:1139226-1141951...,PASS,canonical,E64K,0.000000,0.046895,0.000000,1_1140870_C_T,0,0,0,0,0,0,108,0,46,35,7,0,35,0,1,50
2,TNFRSF18|ENST00000486728.5_1_1:1139226-1140990...,TNFRSF18|ENST00000486728.5_1_1:1139226-1140990...,PASS,canonical_extended,E41K,0.000000,0.051843,0.000000,1_1140870_C_T,0,0,0,0,0,0,108,0,46,35,7,0,35,0,1,50
3,TNFRSF18|ENST00000379265.5_1_1:1139226-1141951...,TNFRSF18|ENST00000379265.5_1_1:1139226-1141951...,PASS,canonical,E64K,0.000000,0.048298,0.000000,1_1140870_C_T,0,0,0,0,0,0,108,0,46,35,7,0,35,0,1,50
4,C1QTNF12|ENST00000330388.2_1_1:1178525-1179833...,C1QTNF12|ENST00000330388.2_1_1:1178525-1179833...,PASS,within,A62T,0.000000,0.000000,0.000000,1_1179460_C_T,0,1,0,0,0,0,49,0,14,17,1,1,44,0,0,40
5,C1QTNF12|ENST00000468365.1_1_1:1178822-1179616...,C1QTNF12|ENST00000468365.1_1_1:1178822-1179616...,PASS,noncoding_other,A53T,0.000000,0.000000,0.000000,1_1179460_C_T,0,1,0,0,0,0,49,0,14,17,1,1,44,0,0,40
6,ATAD3C|ENST00000378785.6_1_1:1386063-1403907:+...,ATAD3C|ENST00000378785.6_1_1:1386063-1403907:+...,PASS,canonical,R62C,0.129308,0.027498,0.062759,1_1387776_C_T,127,0,6,0,24,0,464,0,160,134,224,131,34,0,0,50
7,GABRD|ENST00000640892.1_1_1:1959375-1961718:+|...,GABRD|ENST00000640892.1_1_1:1959375-1961718:+|...,PASS,three_prime_overlap,V4M,0.000000,0.000000,0.000000,1_1959385_G_A,0,0,0,0,0,0,5,0,0,0,0,0,39,0,0,30
8,TTC34|ENST00000401095.8_3_1:2572809-2706156:-|...,TTC34|ENST00000401095.8_3_1:2572809-2706156:-|...,PASS,canonical,P248S,0.000000,0.000000,0.000000,1_2576948_G_A,0,0,0,0,0,0,117,0,46,53,0,0,43,0,0,32
9,PRDM16|ENST00000509860.1_2_1:3313096-3350372:+...,PRDM16|ENST00000509860.1_2_1:3313096-3350372:+...,PASS,canonical_truncated,D409N,0.038453,0.042249,0.000000,1_3328601_G_A,0,2,0,1,0,1,115,0,28,46,25,113,36,0,0,32


In [48]:
len(out)

7021

In [49]:
PASS = out[out['variant_qc'] == 'PASS']

In [51]:
def sum_rpf(row):
    row['wt_rpf_sum'] = int(row['mel11.1_wt_cov']) + int(row['mel11.3_wt_cov']) + int(row['mel11.4_wt_cov'])
    row['mut_rpf_sum'] = int(row['mel11.1_mut_cov']) + int(row['mel11.3_mut_cov']) + int(row['mel11.4_mut_cov'])
    return(row)

PASS = PASS.apply(sum_rpf, axis=1)

In [55]:
PASS_dedup = PASS.drop_duplicates(subset='mut_loc')

In [52]:
PASS.to_csv('../snv_coverage/snv_coverage_across_samples_PASS.txt', sep='\t', header=True, index=False)